In [1]:
# Imports would go here

# Test NMF and Display topics better

In [139]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("")
        print(f"Topic {topic_idx}")
        print("TOP WORDS:", " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print("DOC:", documents[doc_index][:100])

dataset = df.reset_index()
documents = dataset.body

no_features = 700

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, ngram_range=(1,3), stop_words=stops)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CustomVectorizer(# strip_accents='ascii',
                               max_df=0.95,
                               min_df=2,
                               stop_words = stops,
                               max_features=no_features,
                               ngram_range=(1,3)
                              )
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [143]:
lda_model = LatentDirichletAllocation(n_components=16, max_iter=10, n_jobs=-1, learning_method='batch', learning_offset=50.,random_state=0).fit(tf)

In [142]:
# no_topics = 30

for no_topics in range(16, 19, 1):

    # Run NMF
#     nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
#     nmf_W = nmf_model.transform(tfidf)
#     nmf_H = nmf_model.components_

    # Run LDA
    lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=10, n_jobs=-1, learning_method='batch', learning_offset=50.,random_state=0).fit(tf)
    lda_W = lda_model.transform(tf)
    lda_H = lda_model.components_

    no_top_words = 14
    no_top_documents = 8
#     print("")
#     print(f"{no_topics} TOPICS for TF-IDF and NMF RESULTS")
#     print("")
#     display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)
    print("")
    print(f"{no_topics} TOPICS for LDA and DTM RESULTS")
    print("")
    display_topics(lda_H, lda_W, tf_feature_names, documents, no_top_words, no_top_documents)


16 TOPICS for LDA and DTM RESULTS


Topic 0
TOP WORDS: wade shoot ever heat pass post alreadi bosh seen hell time point pleas thing
DOC: i'm out of words number is one of the greatest assists i've ever seen just the difficulty to pass it
DOC: isn't that mostly partially because lebron hasn't been shooting jump shots as often hard to shoot of
DOC: in a lot fewer games too if it already wasn't known he is the best shot blocking guard ever
DOC: lebrons passing ability and court vision is definitely his biggest strength on the court and he will
DOC: clearly a foul but oh my god that's one of the funniest and most ridiculous sells i've ever seen lma
DOC: that's wild i can't recall ever seeing the defender on a on break sticking with the cutter to deny t
DOC: this is not a good thing bosh did this for us with the raptors and dude just slowed down hes one of 
DOC: the least athletic block i've ever seen

Topic 1
TOP WORDS: give shot point mani anyon guard wonder definit second reason chanc n

In [231]:
ms = MeanShift(bin_seeding=True, cluster_all=False, n_jobs=-1)
ms_model = ms.fit(lda_W)

In [233]:
ms_labels = ms.labels_
# Number of clusters in labels, ignoring noise if present.
ms_n_clusters_ = len(set(ms_labels)) - (1 if -1 in ms_labels else 0)
print('Estimated number of clusters: %d' % ms_n_clusters_)

unique, counts = np.unique(ms_labels, return_counts=True)
print('counts')
print(np.asarray((unique, counts)).T)

Estimated number of clusters: 1
counts
[[   -1  6335]
 [    0 39322]]


In [9]:
# DBSCAN MODEL
db = None
db = DBSCAN(eps=0.01, min_samples=25, metric='cosine', leaf_size=10, n_jobs=-1).fit(lda_model)

labels = db.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Estimated number of clusters: %d' % n_clusters_)

unique, counts = np.unique(labels, return_counts=True)
print('counts')
print(np.asarray((unique, counts)).T)

Estimated number of clusters: 2
counts
[[   -1  6116]
 [    0 36081]
 [    1  3474]]
